In [ ]:
import numpy as np
import scipy.io
import matplotlib.pyplot as plt

import os
import sys
sys.path.append('../code')

from pomegranate import HiddenMarkovModel, DiscreteDistribution, NormalDistribution